In [78]:
import pandas as pd
import numpy as np
import spacy
import random

from spacy.util import minibatch, compounding

import src.train as train
import re


import matplotlib
import matplotlib.pyplot as plt
import seaborn as sns

# Cleaning Dataset

In [2]:
df = pd.read_csv('input/#jokermovie.csv')

In [3]:
dfText = df[['id', 'text']]

In [4]:
dfText.head()

,id,text
0,1180633752885743616,People at the cinema thinking they got the #Jo...
1,1180633751744913408,I have a new favorite joker @jokermovie #Joker...
2,1180633723932483584,"⭐⭐⭐⭐⭐ Performance, what a movie #JokerMovie pi..."
3,1180633642198077441,#JokerMovie was absolutely amazing and ended w...
4,1180633577693880321,I just want to give both Arthur Fleck and Gary...


# Create Training Data
Characters, Talent, plot, soundtrack, tickets, hype, movie

In [5]:
categories = {
    'CHARACTERS': ['character', 'characters', 'Arthur Fleck', 'Murray Franklin', 'Sophie Dumond', 'Penny Fleck', 'mother', 'mum'],
    'TALENT': ['actor', 'actress', 'actors', 'talent', 'director', 'Joaquin Phoenix', '#JoaquinPhoenix', 'Robert De Niro', 'Zazie Beetz', 'Frances Conroy', 'Todd Philips'],
    'PLOT': ['plot', 'frame', 'scene', 'scenes'],
    'TICKETS': ['ticket', 'tickets', 'presale'],
    'SOUNDTRACK': ['music', 'song'],
    'SPECIAL EFFECTS': ['FX', 'visual effects', 'graphics', 'postproduction'],
    'COSTUME DESIGN': ['costum', 'costumes', 'costume designer'],
    'HYPE': ['hype', 'hyyyype', 'amazing', 'awesome', 'desire', 'desires', 'premiere', 'Cant wait'],
    'BOX OFFICE': ['box office'],
    'REVIEWS': ['review', 'reviews'],
    'TRAILER':['trailer', 'trailers']
}

In [16]:
TRAIN_DATA = [
    (
        "People at the cinema thinking they got the #JokerMovie  plot like: pic.twitter.com/FULbcu3SBj",
        {"entities": [(56, 60, "PLOT")]},),
     (
        "I just want to give both Arthur Fleck and Gary a hug. #JokerMovie #Joker #JokerFilm",
        {"entities": [(25, 37, "CHARACTERS")]},
    ),
    
    (
        "These 3 actors BETTER get nominated or even win an Oscar. #AvengersEndgame #JokerMovie #ITChapterTwopic.twitter.com/Sd9Ele81wz",
        {"entities": [(8, 14, "TALENT")]},
    )
    ]

In [38]:
TRAIN_DATA = train.trainData(dfText, categories)
print(training)

[('Who would go see a violent, scary #movie like #JokerMovie @jokermovie and why are #films like this produced? It can encourage violence.  #BanViolentFilms  #RememberAurora  We need more uplifting films like @MrRogersMovie @tomhanks', {'entities': []}), ('#JokerMovie Add it is #Jokerit another #clownstar  horror story or #shooting tales by some guilty criminal guys !', {'entities': []}), ('Folks, #JokerMovie doesn’t come out until next week, but here’s an early preview. https://twitter.com/FiteTV/status/1178089904439476224\xa0…', {'entities': [(74, 80, 'REVIEWS')]}), ("On my way to see #joker. Can't wait to see it.\n\nHave read the snowflakes sites (@voxdotcom & @DEADLINE 2 name two)  scare tactics to keep ppl away. \n\nIts almost as if sites like these want something bad to happen, at a joker screening.  \n\n #joker2019   #jokermovie #rottentomatos", {'entities': []}), ('EXCLUSIVE! interviews at the "Joker" World Premiere Joaquin Phoenix & cast! https://youtu.be/qozwerngX6s\xa0  @Joa

# Creating new spaCy model

In [39]:
from __future__ import unicode_literals, print_function

import plac
import random
from pathlib import Path
import spacy
from spacy.util import minibatch, compounding





@plac.annotations(
    model=("Model name. Defaults to blank 'en' model.", "option", "m", str),
    output_dir=("Optional output directory", "option", "o", Path),
    n_iter=("Number of training iterations", "option", "n", int),
)
def main(model=None, output_dir=None, n_iter=100):
    """Load the model, set up the pipeline and train the entity recognizer."""
    if model is not None:
        nlp = spacy.load(model)  # load existing spaCy model
        print("Loaded model '%s'" % model)
    else:
        nlp = spacy.blank("en")  # create blank Language class
        print("Created blank 'en' model")

    # create the built-in pipeline components and add them to the pipeline
    # nlp.create_pipe works for built-ins that are registered with spaCy
    if "ner" not in nlp.pipe_names:
        ner = nlp.create_pipe("ner")
        nlp.add_pipe(ner, last=True)
    # otherwise, get it so we can add labels
    else:
        ner = nlp.get_pipe("ner")

    # add labels
    for _, annotations in TRAIN_DATA:
        for ent in annotations.get("entities"):
            ner.add_label(ent[2])

    # get names of other pipes to disable them during training
    pipe_exceptions = ["ner", "trf_wordpiecer", "trf_tok2vec"]
    other_pipes = [pipe for pipe in nlp.pipe_names if pipe not in pipe_exceptions]
    with nlp.disable_pipes(*other_pipes):  # only train NER
        # reset and initialize the weights randomly – but only if we're
        # training a new model
        if model is None:
            nlp.begin_training()
        for itn in range(n_iter):
            random.shuffle(TRAIN_DATA)
            losses = {}
            # batch up the examples using spaCy's minibatch
            batches = minibatch(TRAIN_DATA, size=compounding(4.0, 32.0, 1.001))
            for batch in batches:
                texts, annotations = zip(*batch)
                nlp.update(
                    texts,  # batch of texts
                    annotations,  # batch of annotations
                    drop=0.5,  # dropout - make it harder to memorise data
                    losses=losses,
                )
            print("Losses", losses)

    # test the trained model
    for text, _ in TRAIN_DATA:
        doc = nlp(text)
        print("Entities", [(ent.text, ent.label_) for ent in doc.ents])
        print("Tokens", [(t.text, t.ent_type_, t.ent_iob) for t in doc])

    # save model to output directory
    if output_dir is not None:
        output_dir = Path(output_dir)
        if not output_dir.exists():
            output_dir.mkdir()
        nlp.to_disk(output_dir)
        print("Saved model to", output_dir)

        # test the saved model
        print("Loading from", output_dir)
        nlp2 = spacy.load(output_dir)
        for text, _ in TRAIN_DATA:
            doc = nlp2(text)
            print("Entities", [(ent.text, ent.label_) for ent in doc.ents])
            print("Tokens", [(t.text, t.ent_type_, t.ent_iob) for t in doc])


In [41]:
main(output_dir='output')

Created blank 'en' model
Losses {'ner': 1261.5985061284323}
Losses {'ner': 109.88038579003471}
Losses {'ner': 124.15215576016368}
Losses {'ner': 156.57347297083047}
Losses {'ner': 108.06906173907011}
Losses {'ner': 87.76115648885683}
Losses {'ner': 93.07801668857235}
Losses {'ner': 97.02200941305874}
Losses {'ner': 68.95360071940975}
Losses {'ner': 63.421014801105116}
Losses {'ner': 66.15394959682988}
Losses {'ner': 49.42838673308983}
Losses {'ner': 60.91109234503314}
Losses {'ner': 49.97882680490092}
Losses {'ner': 40.917436650064474}
Losses {'ner': 43.28155045941898}
Losses {'ner': 39.806531154661954}
Losses {'ner': 41.964906670309844}
Losses {'ner': 23.473779096138063}
Losses {'ner': 31.599027793026444}
Losses {'ner': 26.73476474271833}
Losses {'ner': 39.52592111002518}
Losses {'ner': 23.953925016422296}
Losses {'ner': 16.707435347097302}
Losses {'ner': 25.40497010099357}
Losses {'ner': 14.352688203086151}
Losses {'ner': 24.720028281045963}
Losses {'ner': 30.118962951505214}
Losses 

Entities []
Tokens [('Well', '', 2), ('.', '', 2), ('Here', '', 2), ('goes', '', 2), ('nothing', '', 2), ('.', '', 2), ('Let', '', 2), ('’s', '', 2), ('see', '', 2), ('what', '', 2), ('all', '', 2), ('the', '', 2), ('fuss', '', 2), ('is', '', 2), ('about', '', 2), ('.', '', 2), ('#', '', 2), ('JokerMovie', '', 2), ('pic.twitter.com/RJEEXkQaXK', '', 2)]
Entities []
Tokens [('#', '', 2), ('JokerMovie', '', 2), ('producer', '', 2), ('defends', '', 2), ('movie', '', 2), ('for', '', 2), ('holding', '', 2), ('‘', '', 2), ('mirror', '', 2), ('to', '', 2), ('our', '', 2), ('society', '', 2), ('’', '', 2), (':', '', 2), ('https://nypost.com/2019/09/26/joker-producer-defends-movie-for-holding-mirror-to-our-society/', '', 2), ('\xa0', '', 2), ('…', '', 2)]
Entities []
Tokens [('I', '', 2), ('just', '', 2), ('saw', '', 2), ('#', '', 2), ('JokerMovie', '', 2), ('\n\n', '', 2), ('Did', '', 2), ('your', '', 2), ('theatre', '', 2), ('have', '', 2), ('armed', '', 2), ('security', '', 2), ('like', '', 2

Entities []
Tokens [('#', '', 2), ('JokerMovie', '', 2), ('It', '', 2), ('shows', '', 2), ('how', '', 2), ('everyone', '', 2), ('-', '', 2), ('society', '', 2), (',', '', 2), ('the', '', 2), ('system', '', 2), (',', '', 2), ('bad', '', 2), ('parenting', '', 2), (',', '', 2), ('&', '', 2), ('entertainment', '', 2), ('-', '', 2), ('are', '', 2), ('to', '', 2), ('blame', '', 2), ('for', '', 2), ('the', '', 2), ('mental', '', 2), ('health', '', 2), ('epidemic', '', 2), ('.', '', 2), ('Outside', '', 2), ('of', '', 2), ('maybe', '', 2), ('a', '', 2), ('few', '', 2), ('scenes', '', 2), (',', '', 2), ('I', '', 2), ('felt', '', 2), ('no', '', 2), ('sympathy', '', 2), ('for', '', 2), ('The', '', 2), ('Joker', '', 2), ('.', '', 2), ('Like', '', 2), ('many', '', 2), ('with', '', 2), ('emotional', '', 2), ('struggles', '', 2), (',', '', 2), ('he', '', 2), ('is', '', 2), ("n't", '', 2), ('the', '', 2), ('only', '', 2), ('one', '', 2), ('who', '', 2), ('had', '', 2), ('a', '', 2), ('hand', '', 2), ('

Entities []
Tokens [('Was', '', 2), ('really', '', 2), ('excited', '', 2), ('to', '', 2), ('go', '', 2), ('see', '', 2), ('#', '', 2), ('JokerMovie', '', 2), ('on', '', 2), ('my', '', 2), ('birthday', '', 2), ('.', '', 2), ('Now', '', 2), (',', '', 2), ('some', '', 2), ('fucking', '', 2), ('incels', '', 2), ('have', '', 2), ('ruined', '', 2), ('my', '', 2), ('good', '', 2), ('time', '', 2), ('.', '', 2), ('\n\n', '', 2), ('It', '', 2), ('was', '', 2), ('LITERALLY', '', 2), ('my', '', 2), ('only', '', 2), ('birthday', '', 2), ('plan', '', 2), ('.', '', 2)]
Entities []
Tokens [('#', '', 2), ('JokerMovie', '', 2), ('As', '', 2), ('well', '', 2), ('-', '', 2), ('written', '', 2), ('and', '', 2), ('performed', '', 2), ('as', '', 2), ('it', '', 2), ('is', '', 2), (',', '', 2), ('this', '', 2), ('film', '', 2), ('is', '', 2), ('graphically', '', 2), ('violent', '', 2), ('and', '', 2), ('psychologically', '', 2), ('disturbing', '', 2), ('telling', '', 2), ('the', '', 2), ('dark', '', 2), ('tal

Entities [('#JoaquinPhoenix', 'TALENT')]
Tokens [('And', '', 2), ('the', '', 2), ('Oscar', '', 2), ('goes', '', 2), ('to', '', 2), ('...', '', 2), ('#', 'TALENT', 3), ('JoaquinPhoenix', 'TALENT', 1), ('!', '', 2), ('\n', '', 2), ('Genius', '', 2), ('Performance', '', 2), ('👏', '', 2), ('🏼', '', 2), ('👏', '', 2), ('🏼', '', 2), ('👏', '', 2), ('🏼', '', 2), ('👏', '', 2), ('🏼', '', 2), ('\n\n', '', 2), ('#', '', 2), ('Joker', '', 2), ('#', '', 2), ('JokerMovie', '', 2)]
Entities []
Tokens [('Easily', '', 2), ('the', '', 2), ('best', '', 2), ('film', '', 2), ('this', '', 2), ('year', '', 2), ('.', '', 2), ('Perhaps', '', 2), ('the', '', 2), ('best', '', 2), ('this', '', 2), ('decade', '', 2), ('.', '', 2), ('Still', '', 2), ('collecting', '', 2), ('myself', '', 2), ('but', '', 2), ('you', '', 2), ('need', '', 2), ('to', '', 2), ('go', '', 2), ('see', '', 2), ('the', '', 2), ('@jokermovie', '', 2), ('if', '', 2), ('you', '', 2), ('can', '', 2), ('.', '', 2), ('⭐', '', 2), ('️', '', 2), ('⭐', 

...

Entities [('ticket', 'TICKETS')]
Tokens [('I', '', 2), ('’ve', '', 2), ('already', '', 2), ('reserved', '', 2), ('my', '', 2), ('ticket', 'TICKETS', 3), ('to', '', 2), ('see', '', 2), ('the', '', 2), ('#', '', 2), ('JokerMovie', '', 2), ('this', '', 2), ('weekend', '', 2), ('.', '', 2), ('Since', '', 2), ('I', '', 2), ('’m', '', 2), ('Asian', '', 2), (',', '', 2), ('no', '', 2), ('one', '', 2), ('was', '', 2), ('worried', '', 2), ('about', '', 2), ('me', '', 2), ('being', '', 2), ('inspired', '', 2), ('to', '', 2), ('shoot', '', 2), ('up', '', 2), ('the', '', 2), ('theater', '', 2), (',', '', 2), ('or', '', 2), ('that', '', 2), ('I', '', 2), ('’m', '', 2), ('ANY', '', 2), ('kind', '', 2), ('of', '', 2), ('threat', '', 2), ('at', '', 2), ('all', '', 2), ('.', '', 2), ('#', '', 2), ('SadBecauseItsTrue', '', 2)]
Entities [('Joaquin Phoenix', 'TALENT')]
Tokens [('#', '', 2), ('JokerMovie', '', 2), ('is', '', 2), ('incredible', '', 2), ('.', '', 2), ('Joaquin', 'TALENT', 3), ('Phoenix', 'TA

Entities []
Tokens [('Halftone', '', 2), ('comic', '', 2), ('style', '', 2), ('drawing', '', 2), ('\n', '', 2), ('#', '', 2), ('joker', '', 2), ('#', '', 2), ('manga', '', 2), ('#', '', 2), ('comics', '', 2), ('#', '', 2), ('jokermovie', '', 2), ('#', '', 2), ('drawingpic.twitter.com/hwyhSdLYM7', '', 2)]
Entities []
Tokens [('#', '', 2), ('JokerMovie', '', 2), ('is', '', 2), ('estimated', '', 2), ('to', '', 2), ('have', '', 2), ('$', '', 2), ('100', '', 2), ('Million', '', 2), ('Box', '', 2), ('office', '', 2), ('in', '', 2), ('the', '', 2), ('opening', '', 2), ('weekend', '', 2), ('.', '', 2), ('Buy', '', 2), ('$', '', 2), ('T', '', 2), ('stock', '', 2), ('before', '', 2), ('it', '', 2), ('is', '', 2), ('too', '', 2), ('late.https://twitter.com/jokermovie/status/1177251484213231618', '', 2), ('\xa0', '', 2), ('…', '', 2)]
Entities []
Tokens [('Saw', '', 2), ('a', '', 2), ('dude', '', 2), ('dressed', '', 2), ('up', '', 2), ('as', '', 2), ('the', '', 2), ('Joker', '', 2), ('riding', '',

Entities []
Tokens [('Was', '', 2), ('really', '', 2), ('excited', '', 2), ('to', '', 2), ('go', '', 2), ('see', '', 2), ('#', '', 2), ('JokerMovie', '', 2), ('on', '', 2), ('my', '', 2), ('birthday', '', 2), ('.', '', 2), ('Now', '', 2), (',', '', 2), ('some', '', 2), ('fucking', '', 2), ('incels', '', 2), ('have', '', 2), ('ruined', '', 2), ('my', '', 2), ('good', '', 2), ('time', '', 2), ('.', '', 2), ('\n\n', '', 2), ('It', '', 2), ('was', '', 2), ('LITERALLY', '', 2), ('my', '', 2), ('only', '', 2), ('birthday', '', 2), ('plan', '', 2), ('.', '', 2)]
Entities []
Tokens [('#', '', 2), ('JokerMovie', '', 2), ('As', '', 2), ('well', '', 2), ('-', '', 2), ('written', '', 2), ('and', '', 2), ('performed', '', 2), ('as', '', 2), ('it', '', 2), ('is', '', 2), (',', '', 2), ('this', '', 2), ('film', '', 2), ('is', '', 2), ('graphically', '', 2), ('violent', '', 2), ('and', '', 2), ('psychologically', '', 2), ('disturbing', '', 2), ('telling', '', 2), ('the', '', 2), ('dark', '', 2), ('tal

Entities [('#JoaquinPhoenix', 'TALENT')]
Tokens [('Watchin', '', 2), ("'", '', 2), ('\n', '', 2), ('Joker', '', 2), ('@jokermovie', '', 2), ('\n', '', 2), ('🤡', '', 2), ('🤡', '', 2), ('🤡', '', 2), ('🤡', '', 2), ('🤡', '', 2), ('🤡', '', 2), ('🤡', '', 2), ('🤡', '', 2), ('\n', '', 2), ('🎬', '', 2), ('🎞', '', 2), ('️', '', 2), ('🎬', '', 2), ('🎞', '', 2), ('️', '', 2), ('🎬', '', 2), ('🎞', '', 2), ('️', '', 2), ('🎬', '', 2), ('🎞', '', 2), ('️', '', 2), ('\n\n', '', 2), ('#', '', 2), ('Joker', '', 2), ('#', '', 2), ('JokerMovie', '', 2), ('#', '', 2), ('JokerFilm', '', 2), ('#', '', 2), ('Joker2019', '', 2), ('#', '', 2), ('Jokerthemovie', '', 2), ('#', 'TALENT', 3), ('JoaquinPhoenix', 'TALENT', 1), ('#', '', 2), ('ArthurFleck', '', 2), ('#', '', 2), ('DC', '', 2), ('#', '', 2), ('Batman', '', 2), ('#', '', 2), ('arkhamasylum', '', 2), ('#', '', 2), ('Gotham', '', 2), ('#', '', 2), ('ThomasWayne', '', 2), ('#', '', 2), ('BruceWayne', '', 2), ('#', '', 2), ('DC', '', 2), ('#', '', 2), ('DCUNIVE

Entities []
Tokens [('Put', '', 2), ('on', '', 2), ('a', '', 2), ('happy', '', 2), ('face', '', 2), ('.', '', 2), ('#', '', 2), ('jokermovie', '', 2), ('#', '', 2), ('quicksketch', '', 2), ('#', '', 2), ('ink', '', 2), ('https://www.instagram.com/p/B25LCgjhVf8/?igshid=1hw52rmwudtek', '', 2), ('\xa0', '', 2), ('…', '', 2)]
Entities []
Tokens [('JOKER', '', 2), ('–', '', 2), ('Final', '', 2), ('Trailer', '', 2), ('–', '', 2), ('Warner', '', 2), ('Bros.', '', 2), ('#', '', 2), ('Joker', '', 2), ('#', '', 2), ('JokerMovie', '', 2), ('https://www.youtube.com/watch?v=xRjvmVaFHkk', '', 2), ('\xa0', '', 2), ('…', '', 2)]
Entities []
Tokens [('Alright', '', 2), (',', '', 2), ('#', '', 2), ('JokerMovie', '', 2), ('time', '', 2), ('!', '', 2), ('@JokerMovie', '', 2), ('@thedcuniverse', '', 2), ('#', '', 2), ('DCUniverse', '', 2), ('@IMAX', '', 2), ('#', '', 2), ('imaxpic.twitter.com/9fUMK0qUVw', '', 2)]
Entities []
Tokens [('#', '', 2), ('BigMouth', '', 2), ('Season', '', 2), ('3', '', 2), ('come

In [43]:
nlp = spacy.load('output')

In [53]:
text = dfText['text'][1199]

In [55]:
for t in df['text']:
    doc = nlp(t)
    
    print("Entities", [(ent.text, ent.label_) for ent in doc.ents])
    #print("Tokens", [(t.text, t.ent_type_, t.ent_iob) for t in doc])

Entities [('plot', 'PLOT')]
Entities []
Entities []
Entities [('amazing', 'HYPE')]
Entities [('Arthur Fleck', 'CHARACTERS')]
Entities []
Entities [('Todd Philips', 'TALENT')]
Entities [('#JoaquinPhoenix', 'TALENT')]
Entities []
Entities []
Entities []
Entities [('premiere', 'HYPE')]
Entities []
Entities []
Entities [('character', 'CHARACTERS')]
Entities []
Entities []
Entities []
Entities []
Entities [('Joaquin Phoenix', 'TALENT')]
Entities []
Entities []
Entities []
Entities [('review', 'REVIEWS')]
Entities []
Entities [('#JoaquinPhoenix', 'TALENT')]
Entities []
Entities [('amazing', 'HYPE')]
Entities []
Entities []
Entities []
Entities []
Entities [('Joaquin Phoenix', 'TALENT')]
Entities []
Entities [('ticket', 'TICKETS')]
Entities []
Entities []
Entities []
Entities []
Entities []
Entities [('Joaquin Phoenix', 'TALENT')]
Entities [('#JoaquinPhoenix', 'TALENT')]
Entities []
Entities [('#JoaquinPhoenix', 'TALENT')]
Entities []
Entities []
Entities [('box office', 'BOX OFFICE')]
Entiti

Entities [('Premiere', 'HYPE')]
Entities []
Entities []
Entities []
Entities [('premiere', 'HYPE')]
Entities []
Entities []
Entities []
Entities []
Entities []
Entities []
Entities []
Entities []
Entities [('Joaquin Phoenix', 'TALENT')]
Entities [('#JoaquinPhoenix', 'TALENT')]
Entities []
Entities []
Entities []
Entities []
Entities []
Entities []
Entities []
Entities []
Entities []
Entities []
Entities [('Joaquin Phoenix', 'TALENT')]
Entities []
Entities []
Entities []
Entities []
Entities []
Entities []
Entities []
Entities []
Entities []
Entities []
Entities []
Entities []
Entities []
Entities []
Entities []
Entities []
Entities []
Entities []
Entities []
Entities []
Entities [('Joaquin Phoenix', 'TALENT')]
Entities []
Entities []
Entities []
Entities [('Joaquin Phoenix', 'TALENT')]
Entities [('Joaquin Phoenix', 'TALENT')]
Entities []
Entities []
Entities []
Entities []
Entities []
Entities [('Joaquin Phoenix', 'TALENT')]
Entities []
Entities [('Joaquin Phoenix', 'TALENT')]
Entities

Entities []
Entities []
Entities []
Entities []
Entities []
Entities []
Entities []
Entities []
Entities []
Entities []
Entities []
Entities [('Prey trailer', 'TALENT')]
Entities []
Entities []
Entities []
Entities []
Entities []
Entities []
Entities []
Entities []
Entities []
Entities []
Entities []
Entities []
Entities []
Entities []
Entities []
Entities []
Entities []
Entities []
Entities []
Entities []
Entities []
Entities []
Entities []
Entities []
Entities [('#JoaquinPhoenix', 'TALENT')]
Entities [('joaquin phoenix', 'TALENT')]
Entities []
Entities []
Entities []
Entities []
Entities [('Joaquin Phoenix', 'TALENT')]
Entities []
Entities []
Entities []
Entities []
Entities []
Entities [('Joaquin Phoenix', 'TALENT')]
Entities []
Entities []
Entities []
Entities []
Entities []
Entities []
Entities []
Entities []
Entities [('trailer', 'REVIEWS')]
Entities []
Entities []
Entities []
Entities []
Entities []
Entities []
Entities []
Entities []
Entities [('#JoaquinPhoenix', 'TALENT')]
Ent

Entities [('amazing', 'HYPE'), ('Joaquin Phoenix', 'TALENT')]
Entities []
Entities [('#JoaquinPhoenix', 'TALENT'), ('character', 'CHARACTERS')]
Entities [('Joaquin Pheonix', 'TALENT')]
Entities []
Entities []
Entities []
Entities []
Entities []
Entities []
Entities []
Entities []
Entities []
Entities []
Entities []
Entities [('ticket', 'REVIEWS')]
Entities []
Entities [('Joaquin Phoenix', 'TALENT')]
Entities []
Entities [('Joaquin Phoenix', 'TALENT')]
Entities []
Entities []
Entities []
Entities []
Entities []
Entities []
Entities []
Entities [('review', 'REVIEWS')]
Entities [('premiere', 'HYPE')]
Entities []
Entities [('#JoaquinPhoenix', 'TALENT')]
Entities [('#JoaquinPhoenix', 'TALENT')]
Entities []
Entities [('#JoaquinPhoenix', 'TALENT')]
Entities []
Entities []
Entities []
Entities []
Entities []
Entities [('premiere', 'HYPE')]
Entities [('#JoaquinPhoenix', 'TALENT')]
Entities []
Entities []
Entities []
Entities [('Joaquin Phoenix', 'TALENT')]
Entities []
Entities []
Entities []
En

Entities []
Entities []
Entities []
Entities []
Entities []
Entities []
Entities []
Entities []
Entities []
Entities []
Entities []
Entities []
Entities [('ruin America', 'TALENT')]
Entities []
Entities []
Entities []
Entities [('❤️#JoaquinPhoenix', 'TALENT')]
Entities []
Entities []
Entities [('character', 'CHARACTERS')]
Entities []
Entities []
Entities []
Entities []
Entities []
Entities []
Entities []
Entities []
Entities [('character', 'CHARACTERS')]
Entities [('Arthur Fleck', 'CHARACTERS')]
Entities [('reward', 'REVIEWS'), ('Joaquin Phoenix', 'TALENT')]
Entities []
Entities []
Entities []
Entities [('Joaquin Phoenix', 'TALENT')]
Entities []
Entities []
Entities []
Entities []
Entities []
Entities []
Entities []
Entities []
Entities [('amazing', 'HYPE')]
Entities [('Joaquin Pheonix', 'TALENT'), ('Arthur Fleck', 'CHARACTERS')]
Entities []
Entities []
Entities []
Entities [('character', 'CHARACTERS'), ('Joaquin Phoenix', 'TALENT')]
Entities []
Entities []
Entities []
Entities []
Enti

...

Entities []
Entities []
Entities []
Entities []
Entities []
Entities []
Entities []
Entities [('review', 'REVIEWS')]
Entities [('Joaquin Phoenix', 'TALENT')]
Entities [('Joaquin Phoenix', 'TALENT')]
Entities []
Entities [('Joaquin Phoenix', 'TALENT'), ('character', 'CHARACTERS')]
Entities []
Entities []
Entities []
Entities []
Entities []
Entities []
Entities [('amazing', 'HYPE')]
Entities []
Entities []
Entities [('#JoaquinPhoenix', 'TALENT')]
Entities []
Entities [('#معصتي_راحتي', 'TALENT')]
Entities []
Entities []
Entities []
Entities []
Entities []
Entities []
Entities []
Entities []
Entities []
Entities []
Entities []
Entities []
Entities []
Entities []
Entities [('amazing', 'HYPE')]
Entities []
Entities []
Entities [('character', 'CHARACTERS'), ('. Phoenix', 'TALENT')]
Entities [('#JoaquinPhoenix', 'TALENT')]
Entities []
Entities []
Entities []
Entities []
Entities []
Entities []
Entities []
Entities []
Entities []
Entities []
Entities []
Entities []
Entities []
Entities [('and P

Entities []
Entities []
Entities []
Entities []
Entities [('Joaquin Phoenix', 'TALENT')]
Entities []
Entities [('#JoaquinPhoenix', 'TALENT')]
Entities [('ticket', 'TRAILER')]
Entities []
Entities []
Entities []
Entities [('#JoaquinPhoenix', 'TALENT')]
Entities []
Entities []
Entities []
Entities []
Entities [('Joaquin Phoenix', 'TALENT')]
Entities []
Entities []
Entities []
Entities []
Entities []
Entities []
Entities []
Entities []
Entities []
Entities []
Entities []
Entities []
Entities []
Entities []
Entities []
Entities []
Entities []
Entities []
Entities []
Entities []
Entities [('Joachim Phoenix', 'TALENT')]
Entities []
Entities [('#JoaquinPhoenix', 'TALENT')]
Entities []
Entities [('amazing', 'HYPE')]
Entities []
Entities []
Entities []
Entities []
Entities []
Entities []
Entities []
Entities []
Entities []
Entities []
Entities [('#JoaquinPhoenix', 'TALENT')]
Entities []
Entities []
Entities []
Entities []
Entities []
Entities []
Entities [('#JoaquinPhoenix', 'TALENT')]
Entities

Entities []
Entities [('#JoaquinPhoenix', 'TALENT')]
Entities []
Entities []
Entities []
Entities []
Entities []
Entities []
Entities [('amazing', 'HYPE')]
Entities []
Entities []
Entities [('Joaquin Phoenix', 'TALENT')]
Entities []
Entities []
Entities []
Entities [('amazing', 'HYPE')]
Entities []
Entities []
Entities []
Entities [('Joaquin Phoenix', 'TALENT')]
Entities []
Entities [('#JoaquinPhoenix', 'TALENT'), ('#JoaquinPhoenix', 'TALENT'), ('Arthur Fleck', 'CHARACTERS')]
Entities []
Entities []
Entities []
Entities []
Entities []
Entities []
Entities [('box office', 'BOX OFFICE')]
Entities []
Entities []
Entities [('#JoaquinPhoenix', 'TALENT')]
Entities []
Entities []
Entities [('Joaquim Phoenix', 'TALENT')]
Entities []
Entities []
Entities [('box office', 'BOX OFFICE')]
Entities []
Entities []
Entities []
Entities []
Entities []
Entities []
Entities []
Entities []
Entities []
Entities []
Entities []
Entities []
Entities []
Entities []
Entities []
Entities []
Entities []
Entities 

Entities [('#JoaquinPhoenix', 'TALENT')]
Entities []
Entities []
Entities []
Entities []
Entities [('amazing', 'HYPE'), ('#JoaquinPhoenix', 'TALENT')]
Entities []
Entities []
Entities []
Entities [('Joaquin Phoenix', 'TALENT')]
Entities [('Joaquin Phoenix', 'TALENT')]
Entities []
Entities []
Entities []
Entities [('Joaquin Phoenix', 'TALENT')]
Entities []
Entities []
Entities []
Entities [('#JoaquinPhoenix', 'TALENT')]
Entities []
Entities [('Joaquin Phoenix', 'TALENT')]
Entities []
Entities []
Entities []
Entities []
Entities []
Entities []
Entities []
Entities []
Entities []
Entities []
Entities [('Joaquin Phoenix', 'TALENT')]
Entities []
Entities []
Entities []
Entities []
Entities []
Entities []
Entities []
Entities [('review', 'REVIEWS'), ('premiere', 'HYPE')]
Entities []
Entities []
Entities []
Entities []
Entities [('Joaquin Phoenix', 'TALENT')]
Entities [('character', 'CHARACTERS'), ('character', 'CHARACTERS')]
Entities []
Entities [('#JoaquinPhoenix', 'TALENT')]
Entities []
En

Entities []
Entities []
Entities []
Entities []
Entities []
Entities []
Entities []
Entities [('review', 'REVIEWS')]
Entities []
Entities []
Entities []
Entities []
Entities []
Entities []
Entities []
Entities [('Joaquin Phoenix', 'TALENT')]
Entities []
Entities []
Entities []
Entities []
Entities []
Entities []
Entities [('Joaquin Phoenix', 'TALENT')]
Entities []
Entities [('amazing', 'HYPE')]
Entities [('#JoaquinPhoenix', 'TALENT')]
Entities []
Entities []
Entities []
Entities []
Entities []
Entities []
Entities []
Entities []
Entities []
Entities []
Entities [('#JoaquinPhoenix', 'TALENT')]
Entities []
Entities []
Entities []
Entities [('Joaquin Phoenix', 'TALENT')]
Entities []
Entities []
Entities []
Entities []
Entities []
Entities [('#JoaquinPhoenix', 'TALENT')]
Entities [('review', 'REVIEWS'), ('#JoaquinPheonix', 'TALENT')]
Entities []
Entities []
Entities [('character', 'CHARACTERS')]
Entities [('#JoaquinPhoenix', 'TALENT')]
Entities [('review', 'REVIEWS'), ('Joaquin Phoenix', '

In [73]:
df = df[['id', 'author_id','author_name', 'author_handler', 'author_avatar','lang', 'type', 'text', 'date', 'mention_ids']]

In [74]:
for index, row in df.iterrows():
    doc = nlp(row['text'])
    for ent in doc.ents:
        df.loc[index, 'category'] = [(ent.label_)]

/Library/Frameworks/Python.framework/Versions/3.8/lib/python3.8/site-packages/pandas/core/indexing.py:376: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self.obj[key] = _infer_fill_value(value)
/Library/Frameworks/Python.framework/Versions/3.8/lib/python3.8/site-packages/pandas/core/indexing.py:494: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self.obj[item] = s


In [75]:
df.head()

,id,author_id,author_name,author_handler,author_avatar,lang,type,text,date,mention_ids,category
0,1180633752885743616,2866588387,BagMan,The_BagMan_,https://pbs.twimg.com/profile_images/123315125...,en,original,People at the cinema thinking they got the #Jo...,2019-10-05T23:59:56.000Z,[],PLOT
1,1180633751744913408,77537588,david,bizzarr1000,https://pbs.twimg.com/profile_images/122677952...,en,original,I have a new favorite joker @jokermovie #Joker...,2019-10-05T23:59:56.000Z,"[""1038128227485532160""]",NaN
2,1180633723932483584,701525750,Ijaz Moin,ijjy7,https://pbs.twimg.com/profile_images/866198526...,en,original,"⭐⭐⭐⭐⭐ Performance, what a movie #JokerMovie pi...",2019-10-05T23:59:49.000Z,[],NaN
3,1180633642198077441,437109027,Brandon Denson,BrandonDeShane2,https://pbs.twimg.com/profile_images/119195475...,en,original,#JokerMovie was absolutely amazing and ended w...,2019-10-05T23:59:30.000Z,[],HYPE
4,1180633577693880321,1644317684,Adam,Klakuladd,https://pbs.twimg.com/profile_images/123146325...,en,original,I just want to give both Arthur Fleck and Gary...,2019-10-05T23:59:14.000Z,[],CHARACTERS


In [76]:
df.to_csv('predicted.csv')

FileNotFoundError: [Errno 2] File b'src/predicted.csv' does not exist: b'src/predicted.csv'